In [1]:
from src.graph import graph_loader, create_polarized_graph, spectral_bipartition_coloring, random_color_graph
from src.seed import seed_degree, seed_random 
import networkx as nx
from icm_diffusion import simulate_diffusion_ICM
import pandas as pd
import random

In [6]:
def edge_addition_adamic_adar(G, seeds, k):
    graph = G.copy()

    # Convert to undirected graph for Adamic-Adar calculation
    undirected_graph = graph.to_undirected()

    for seed in seeds:
        # Compute Adamic-Adar index for pairs involving the seed node
        adamic_adar_scores = list(nx.adamic_adar_index(undirected_graph, [(seed, n) for n in undirected_graph.nodes if n != seed]))

        # Sort the scores in descending order
        adamic_adar_scores.sort(key=lambda x: x[2], reverse=True)

        # Add edges to the graph as a connection from the seed to the top k nodes
        for i in range(min(k, len(adamic_adar_scores))):
            target_node = adamic_adar_scores[i][1]
            graph.add_edge(seed, target_node)

    return graph

def edge_addition_preferential_attachment(G, seeds, k):
    graph = G.copy()
    
    for seed in seeds:
        # Calculate the degree of all nodes in the graph
        node_degrees = dict(graph.degree())
        
        # Generate the cumulative distribution of degrees for random selection
        nodes, degrees = zip(*node_degrees.items())
        total_degree = sum(degrees)
        cumulative_distribution = [sum(degrees[:i+1]) / total_degree for i in range(len(degrees))]

        # Add edges from the seed node to k nodes chosen by the preferential attachment rule
        for _ in range(k):
            random_value = random.random()
            for i, cum_dist in enumerate(cumulative_distribution):
                if random_value <= cum_dist:
                    target_node = nodes[i]
                    # Prevent self-loops and duplicate edges
                    if target_node != seed and not graph.has_edge(seed, target_node):
                        graph.add_edge(seed, target_node)
                        break
    
    return graph

# Jaccard Coefficient
def edge_addition_jaccard(G, seeds, k):
    graph = G.copy()
    undirected_graph = graph.to_undirected()

    for seed in seeds:
        jaccard_scores = list(nx.jaccard_coefficient(undirected_graph, [(seed, n) for n in undirected_graph.nodes if n != seed]))
        jaccard_scores.sort(key=lambda x: x[2], reverse=True)

        for i in range(min(k, len(jaccard_scores))):
            target_node = jaccard_scores[i][1]
            graph.add_edge(seed, target_node)

    return graph

# Degree
def edge_addition_degree(G, seeds, k):
    graph = G.copy()

    for seed in seeds:
        nodes_sorted_by_degree = sorted(graph.nodes, key=lambda n: graph.out_degree(n), reverse=True)
        for target_node in nodes_sorted_by_degree[:k]:
            if target_node != seed:
                graph.add_edge(seed, target_node)

    return graph

# Harmonic Centrality (Topk)
def edge_addition_topk(G, seeds, k):
    graph = G.copy()
    harmonic_centralities = nx.harmonic_centrality(graph)

    for seed in seeds:
        nodes_sorted_by_centrality = sorted(harmonic_centralities.items(), key=lambda x: x[1], reverse=True)
        for i in range(min(k, len(nodes_sorted_by_centrality))):
            target_node = nodes_sorted_by_centrality[i][0]
            if target_node != seed:
                graph.add_edge(seed, target_node)

    return graph

# Probabilistic Edge Addition (Prob)
def edge_addition_prob(G, seeds, k):
    graph = G.copy()

    for seed in seeds:
        all_possible_edges = [(seed, n) for n in graph.nodes if n != seed and not graph.has_edge(seed, n)]
        if len(all_possible_edges) == 0:
            continue
        random.shuffle(all_possible_edges)
        selected_edges = random.sample(all_possible_edges, min(k, len(all_possible_edges)))

        for edge in selected_edges:
            graph.add_edge(*edge)

    return graph

# Kempe et al. Seed Selection (KKT)
def edge_addition_kkt(G, seeds, k):
    graph = G.copy()

    for seed in seeds:
        candidates = sorted(graph.nodes, key=lambda n: nx.degree_centrality(graph)[n], reverse=True)
        for target_node in candidates[:k]:
            if target_node != seed:
                graph.add_edge(seed, target_node)

    return graph

# Random
def edge_addition_random(G, seeds, k):
    graph = G.copy()

    available_nodes = [n for n in graph.nodes if n not in seeds]
    selected_nodes = random.sample(available_nodes, min(k, len(available_nodes)))
    
    for seed in seeds:
        for target_node in selected_nodes:
            graph.add_edge(seed, target_node)

    return graph

In [7]:
# Function to evaluate and compare the graph modifications
def evaluate_graph_modifications(G, seeds, k, max_iter):
    # Simulate diffusion on the original graph
    count, count_std, color_count, color_count_std = simulate_diffusion_ICM(G, seeds, 1, max_iter)
    
    # Results for the original graph
    original_results = pd.DataFrame({
        'Metric': ['Count', 'Count Standard Deviation', 'Color Count', 'Color Count Standard Deviation'],
        'Original Graph': [round(count, 3), round(count_std, 3), round(color_count, 3), round(color_count_std, 3)]
    })

    # Define a list of modification functions
    modification_functions = {
        'Adamic Adar': edge_addition_adamic_adar,
        'PrefAtt': edge_addition_preferential_attachment,
        'Jaccard': edge_addition_jaccard,
        'Degree': edge_addition_degree,
        'TopK': edge_addition_topk,
        'Prob': edge_addition_prob,
        'KKT': edge_addition_kkt,
        'Random': edge_addition_random
    }

    combined_results = original_results.copy()

    # Evaluate each graph modification
    for method_name, mod_func in modification_functions.items():
        modified_graph = mod_func(G, seeds, k)
        count, count_std, color_count, color_count_std = simulate_diffusion_ICM(modified_graph, seeds, 1, max_iter)

        adapted_results = pd.DataFrame({
            'Metric': ['Count', 'Count Standard Deviation', 'Color Count', 'Color Count Standard Deviation'],
            f'Adapted Graph {method_name}': [round(count, 3), round(count_std, 3), round(color_count, 3), round(color_count_std, 3)]
        })

        combined_results = pd.merge(combined_results, adapted_results, on='Metric')

    # Get the number of nodes and edges for all graphs
    graph_info = {
        'Metric': ['Number of Nodes', 'Number of Edges'],
        'Original Graph': [G.number_of_nodes(), G.number_of_edges()],
    }
    
    for method_name, mod_func in modification_functions.items():
        modified_graph = mod_func(G, seeds, k)
        graph_info[f'Adapted Graph {method_name}'] = [modified_graph.number_of_nodes(), modified_graph.number_of_edges()]
    
    graph_info_df = pd.DataFrame(graph_info)

    # Combine all results into one DataFrame
    final_results = pd.concat([graph_info_df, combined_results], ignore_index=True)

    # Transpose the DataFrame and set the first row as the header
    final_results = final_results.T
    final_results.columns = final_results.iloc[0]  # Set the first row as the column names
    final_results = final_results.drop(final_results.index[0])  # Drop the first row

    return final_results

In [8]:
G = create_polarized_graph(1000, 0.2, 0.01)
#G = graph_loader('datasets/congress_network/congress.edgelist')
#color the graph
spectral_bipartition_coloring(G)

/Users/quenzer/miniforge3/lib/python3.9/site-packages/sklearn/manifold/_spectral_embedding.py:233: UserWarning: Array is not symmetric, and will be converted to symmetric by average with its transpose.
  adjacency = check_symmetric(adjacency)


In [9]:
seed = seed_degree(G, 100)
k = 15
max_iter = 1000
final_results = evaluate_graph_modifications(G, seed, k, max_iter)
final_results

100%|██████████| 1000/1000 [00:11<00:00, 89.26it/s]

In [ ]:
seed = seed_random(G, 100)
k = 15
max_iter = 1000
final_results = evaluate_graph_modifications(G, seed, k, max_iter)
final_results

100%|██████████| 1000/1000 [00:10<00:00, 96.54it/s]


Metric,Number of Nodes,Number of Edges,Count,Count Standard Deviation,Color Count,Color Count Standard Deviation
Original Graph,1000.0,102395.0,243.79,144.557,5.758,8.98
Adapted Graph Adamic Adar,1000.0,103645.0,254.16,148.614,6.124,9.204
Adapted Graph PrefAtt,1000.0,103894.0,266.455,148.54,13.371,16.809
Adapted Graph Jaccard,1000.0,103662.0,267.616,154.881,6.832,10.332
Adapted Graph Degree,1000.0,103761.0,239.091,138.655,8.518,12.266
Adapted Graph TopK,1000.0,103700.0,246.215,141.712,9.121,12.6
Adapted Graph Prob,1000.0,103895.0,253.201,149.118,13.123,17.841
Adapted Graph KKT,1000.0,103736.0,246.993,147.412,9.493,14.163
Adapted Graph Random,1000.0,103756.0,262.031,148.708,12.152,15.703
